# Проект: Защита персональных данных клиентов
**Dataset Description:**

   В этом проекте нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

**Данные:**

 * `insurance.csv` - персональные данные клиентов страховой компании «Хоть потоп».
 
**Описание полей данных:**

 * `Пол` - пол клиента.
 * `Возраст` - возраст.
 * `Зарплата` - среднемесячная запрлата.
 * `Члены семьи	` - количество членов семьи.
 * `Страховые выплаты` - целевой признак, количество страховых выплат клиенту за последние 5 лет..
 
**Цели:**
 
 * Разработка метода преобразования персональных данных клиентов компании при которов не изменяется качество моделей машинного обучения.
 * Ответ на вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?
 * Проверка метрики `R2` модели машинного обучения на исходных данных и на данных после преобразования.
 
**Структура проекта:**
 
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span></a></span></li></ul></div>
 
**Предмет исследования:**

Персональные данные клиентов страховой компании «Хоть потоп»

**Методы:** которые мы применим в данном исследовании включают в себя: 
* Загрузка и ознакомление с данными
* Matrix
* Machine learning

**Используемые библитеки:** 
* Scikit-learn
* Numpy
* Pandas

## Загрузка данных

In [18]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from numpy import linalg

RANDOM_SEED = 1234

Загрузим данные

In [3]:
df = pd.read_csv('/datasets/insurance.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Ознакомимся с первыми пятью строками датасета

In [5]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Размеры датасета

In [6]:
df.shape

(5000, 5)

Пропущенных значений не обнаруженно

In [7]:
df.isnull().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Мы обнаружили повторяющиеся строки, считаю что задача этого проекта не подразумивает удаление дубликатов, оставляем все как есть. В реальной ситуации, мы проконсультировались бы с ответсвенным за выгрузку данных.

In [8]:
df.loc[df.duplicated(keep=False)].sort_values(by='Зарплата').head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
2955,1,32.0,21600.0,0,0
2988,1,32.0,21600.0,0,0
361,0,50.0,24700.0,1,2
2869,0,50.0,24700.0,1,2
333,0,32.0,25600.0,1,0
4230,0,32.0,25600.0,1,0
1378,0,36.0,26400.0,0,0
2723,0,36.0,26400.0,0,0
1002,1,34.0,26900.0,0,0
1140,1,34.0,26900.0,0,0


Посмотрим на типы данных в столбцах датасета:

In [9]:
df.dtypes

Пол                    int64
Возраст              float64
Зарплата             float64
Члены семьи            int64
Страховые выплаты      int64
dtype: object

In [10]:
df['Возраст'].unique()

array([41., 46., 29., 21., 28., 43., 39., 25., 36., 32., 38., 23., 40.,
       34., 26., 42., 27., 33., 47., 30., 19., 31., 22., 20., 24., 18.,
       37., 48., 45., 44., 52., 49., 35., 56., 65., 55., 57., 54., 50.,
       53., 51., 58., 59., 60., 61., 62.])

In [11]:
df['Зарплата'].unique()[:30]

array([49600., 38000., 21000., 41700., 26100., 41000., 39700., 38600.,
       49700., 51700., 36600., 29300., 39500., 55000., 43700., 23300.,
       48900., 33200., 36900., 43500., 36100., 26600., 48700., 40400.,
       38400., 34600., 34800., 36800., 42200., 46300.])

Преобразуем данные в столбцах `Возраст` и `Зарплата` в тип `int`, диапазоны данных и их точность позволяют нам это сделать

In [12]:
int_col = ['Возраст', 'Зарплата']
df[int_col] = df[int_col].astype('int')

In [13]:
df.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** При умножении матрицы признаков на обратимую матрицу качество линейной регрессии не измениться.

**Обоснование:** 

Пусть $Z$ это матрица признаков $X$ умноженная на обратимую матрицу $P$:

$$Z = XP$$

тогда предсказание:

$$
a_1=Zw_1
$$

задача обучения:

$$
w_1 = \arg\min_w MSE(Zw_1, y)
$$

формула обучения:

$$
w_1 = (Z^T Z)^{-1} Z^T y
$$


т.к $Z = XP$ подставим в правую часть уравнения XP вместо Z:

$$
w_1 = ((XP)^T XP)^{-1} (XP)^T y
$$

руководствуясь тем что $(AB)^{-1} = B^{-1}A^{-1}$ раскроем первое $(XP)^T$

$$
w_1 = (P^{T}X^{T}XP)^{-1}(XP)^{T} y
$$


раскроем $(XP)^T y$

$$
w_1 = (P^{T}X^{T}XP)^{-1}(P)^TX^{T} y
$$


перегруппируем множители в скобках

 $$w1 = (P^{T}(X^{T}X)P)^{-1}P^{T}X^{T} y$$

раскроем скобки в $(P^{T}(X^{T}X)P)^{-1}$


$$
w_1 = P^{-1}(X^{T}X)^{-1}(P^{T})^{-1}P^{T}X^{T} y
$$

т.к. матрица $P$ по условию обратимая то $P^{T}(P^{T})^{-1} = E$

$$
w_1 = P^{-1}(X^{T}X)^{-1}EX^{T} y
$$

$$
w_1 = P^{-1}(X^{T}X)^{-1}X^{T} y
$$

$$
w_1 = P^{-1}w
$$

предсказание по условию равно

$$
a_1=Zw_1
$$

тогда

$$
a_1=XPP^{-1}w
$$

сокращаем

$$
a_1 = Xw = a
$$



Проверим измениться ли качество линейной регрессии при умножении матрицы признаков на обратимую матрицу

In [36]:
np.random.seed(RANDOM_SEED)

features = df.drop('Страховые выплаты', axis=1).values
y = df['Страховые выплаты']
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
n = X.shape[1]
P = np.random.normal(1, 5.5, (n, n))

In [38]:
determinant = linalg.det(P)
print(f'Детерминант матрицы P = {determinant}')
P_inv = np.linalg.inv(P)
print(f'P_inv матрица: {P_inv}')

Детерминант матрицы P = 76688.75261111304
P_inv матрица: [[ 0.04064709  0.16747412 -0.00197484 -0.09721511  0.15602548]
 [-0.08985288 -0.05140661  0.07233381  0.09286481 -0.01104701]
 [ 0.02891719 -0.09860671  0.05134667  0.09477012 -0.03363272]
 [-0.00745966  0.0278464  -0.02424139  0.04898473  0.077699  ]
 [-0.0314162  -0.00290636  0.02187536 -0.01971421  0.09021618]]


Базовая модель обученная на матрице признаков без преобразования

In [39]:
model = LinearRegression()

model.fit(X, y)
score = r2_score(y, model.predict(X))

print(f'Точность модели до умножения матрицы признаков на обратимую матрицу составила: {score}')
coef1 = model. coef_
print(model. coef_)

Точность модели до умножения матрицы признаков на обратимую матрицу составила: 0.42494550308169177
[ 0.00000000e+00  7.92580563e-03  3.57083050e-02 -1.70081903e-07
 -1.35676627e-02]


Модель обученная на матрице признаков умноженных на целое число

In [40]:
model.fit(X*2, y)
score = r2_score(y, model.predict(X*2))

print(f'Точность модели после умножения матрицы признаков на целое число: {score}')
coef3 = model. coef_
print(model. coef_)

Точность модели после умножения матрицы признаков на целое число: 0.42494550308169177
[ 0.00000000e+00  3.96290282e-03  1.78541525e-02 -8.50409514e-08
 -6.78383135e-03]


На вид явная линейная зависимость в изменяется коэффициетах регресии модели

In [41]:
coef1/coef3

/tmp/ipykernel_31/676100693.py:1: RuntimeWarning: invalid value encountered in true_divide
  coef1/coef3


array([nan,  2.,  2.,  2.,  2.])

Модель обученная на матрице признаков умноженных на обратимую матрицу

In [42]:
Z = X @ P

model.fit(Z, y)
score = r2_score(y, model.predict(Z))

print(f'Точность модели после умножения матрицы признаков на обратимую матрицу составила: {score}')
print(model. coef_)
coef2 = model. coef_

Точность модели после умножения матрицы признаков на обратимую матрицу составила: 0.4249451457251834
[-4947314.68696796 10936341.65052025 -3519623.12498367   907944.35763356
  3823787.16459736]


Мы не онаружили линейной зависимости между коэфициентами регрессии модели

In [43]:
coef1/coef2

array([-0.00000000e+00,  7.24721839e-10, -1.01454911e-08, -1.87326350e-13,
       -3.54822643e-09])

Вывод: При умножении матрицы признаков на обратимую матрицу качество линейной регрессии не изменяется, изменяется коэффициеты регресии модели - это говорит нам о том что между первоначальными данными и данными после преобразования отсутвует линейная зависимость.

## Алгоритм преобразования

**Алгоритм**

Для защиты персональных данных клиентов страховой компании «Хоть потоп» будем использовать метод умножения матрицы признаков на обратимую случайно сгенерированную матрицу `P` размерностью матрицы `X.shape[1] на X.shape[1]` Удобство этого метода в том что после преобразования данные нелинейно изменяются и восстановить первоначальные данные можно умножив на обратную матрицу преобразования.

**Обоснование**

Обозначения:

- $X$ — матрица признаков
- $P$ — обратимая матрица размерностью `X.shape[1]` на `X.shape[1]`

Докажем что первоначальная матрица признаков $Х$ равна произведению матрицы признаков $Х$ на обратимую матрицу $P$ и на обратную матрицу $P^{-1}$:

$$
X = XPP^{-1}
$$

 Произведение обратимой матрицы на обратную к ней равно единичной матрице

$$
AB = BA = E
$$

$PP^{-1}$ можем заменить на $E$

$$
X = X(PP^{-1})
$$

Тогда

$$
X = XE
$$

В итоге

$$
X = X
$$

In [44]:
features = df.drop('Страховые выплаты', axis=1)
X = features
n = X.shape[1]
y = df['Страховые выплаты']
P = np.random.randint(1, 10, (n,n))

Первоначальные данные:

In [45]:
X.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


Умножение на обратимую матрицу:

In [46]:
Z = X @ P

Данные защифрованы

In [47]:
Z[:10]

,0,1,2,3
0,297818,248056,49857,446497
1,228239,190053,38285,342099
2,126145,105029,21174,189058
3,250323,208535,41844,375356
4,156744,130536,26270,234964
5,246237,205065,41278,369108
6,238417,198561,39954,357400
7,231765,193061,38788,347486
8,298393,248551,49927,447387
9,310373,258547,51903,465379


Обратное преобразование

In [48]:
t = Z @ np.linalg.inv(P) 
t.round(1).astype(int).head()

,0,1,2,3
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


## Проверка алгоритма

Для проверки будем использовать модель линейной регрессии библиотеки Scikit-learn, для оценки - метрику R2.

Разделим наши данные на тренировочную и тестовую выборки в отношении 0.75 к 0.25 от всего объема данных:

In [24]:
target = df['Страховые выплаты']
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_SEED)

Обучение и проверка модели на данных без преобразования

In [25]:
model.fit(features_train, target_train)
score = r2_score(target_test, model.predict(features_test))
print("R2 =", score)

R2 = 0.4249620550874137


Обучение и проверка модели на данных с преобразованием

In [26]:
features_train, features_test, target_train, target_test = train_test_split(
    Z, target, test_size=0.25, random_state=RANDOM_SEED)

model.fit(features_train, target_train)
score = r2_score(target_test, model.predict(features_test))
print("R2 =", score)

R2 = 0.424962055087424


Точность не изменна

## Вывод

Разработка метода преобразования персональных данных клиентов компании при которов не изменяется качество моделей машинного обучения.

Ответ на вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?
Проверка метрики R2 модели машинного обучения на исходных данных и на данных после преобразования.

В данном проекте мы изучили влияние преобразования признаков на качество модели линейной регрессии при умножении последних на обратимую матрицу.

Для начала мы загрузили данные клиентов компании, ознакомились с ними, проверили на аномальные значения и привели к нужным типам данных. 

После ответили на первый вопрос: При умножении признаков на обратимую матрицу как измениться качество линейной регрессии?
Ответ: При умножении матрицы признаков на обратимую матрицу качество линейной регрессии не измениться.

Затем мы проанализировали регрисионные коэфициенты моделей при преобразовании и пришли к выводу:

При линейном преобразовании признаков, наблюдается линейное изменени коэфициентов модели. Этот метод защиты персональных данных нам не подходит: т.к остается линейная зависимость в измененых данных и они недостаточно защищены.

При умножении персональных данных на обратимую матрицу, данные изменяются нелинейно и точность модели линейной регресии остается преждней. Но коэфициента модели указывают на нелинейное преобразование: [ 0.00613887  0.0007744   0.00107731 -0.0014413  -0.00511657]

Мы определились с алгоритмом преобразования и в четвертой части нашего проекта сравнили метрики `R2` модели линейной регрессии на базовых данных и на преобразованных. В данном случаем метрика осталась постоянной даже после преобразования.

P.S. небольшие различия в метриках моделей особенность вычисления дробных чисел в Phyton, X Float <> X Float